In [1]:
import clickhouse_connect

client = clickhouse_connect.get_client(host='localhost', username='default')

In [2]:
client.command('CREATE DATABASE IF NOT EXISTS dev_test')
client.database = 'dev_test'

client.command("""DROP TABLE IF EXISTS calls""")
client.command("""DROP TABLE IF EXISTS objects""")

client.command("""
    CREATE TABLE IF NOT EXISTS calls (
        id String NOT NULL,
        payload_dump String NOT NULL
    ) ENGINE = MergeTree()
    ORDER BY id;
""")

client.command("""
    CREATE TABLE IF NOT EXISTS objects (
        id String NOT NULL,
        payload_dump String NOT NULL
    ) ENGINE = MergeTree()
    ORDER BY id;
""")

In [3]:
import json
import uuid
import pandas as pd

def insert_call(payload: dict) -> str:
    insert_id = str(uuid.uuid4())
    payload_dump = json.dumps(payload)
    client.insert("calls", data=[[insert_id, payload_dump]], column_names=['id', 'payload_dump'])
    return insert_id

def insert_object(payload: dict) -> str:
    insert_id = str(uuid.uuid4())
    payload_dump = json.dumps(payload)
    client.insert("objects", data=[[insert_id, payload_dump]], column_names=['id', 'payload_dump'])
    return insert_id

def get_calls() -> pd.DataFrame:
    res = client.query_df("SELECT * FROM calls")
    return res

def get_objects() -> pd.DataFrame:
    res = client.query_df("SELECT * FROM objects")
    return res

def object_ref_from_id(object_id: str) -> str:
    return "ref://objects/" + object_id

def insert_ref_object(payload: dict) -> str:
    object_id = insert_object(payload)
    return object_ref_from_id(object_id)


In [4]:
def get_calls_user_perspective() -> pd.DataFrame:
    calls = get_calls().to_dict(orient='records')
    calls = list(calls)
    for call in calls:
        call['payload'] = json.loads(call.pop('payload_dump'))

    objects = get_objects().to_dict(orient='records')
    objects = list(objects)
    for obj in objects:
        obj['payload'] = json.loads(obj.pop('payload_dump'))

    def resolve_ref(ref: str) -> dict:
        if ref.startswith('ref://objects/'):
            object_id = ref[len('ref://objects/'):]
            for obj in objects:
                if obj['id'] == object_id:
                    return obj['payload']
            return None
        else:
            return None
        
    def resolve_dict(d: dict, path=None) -> dict:
        resolved_paths = {}
        if path is None:
            path = []
        for k, v in d.items():
            if isinstance(v, dict):
                res, paths = resolve_dict(v, path + [k])
                resolved_paths.update(paths)
                d[k] = res
            elif isinstance(v, str):
                if v.startswith('ref://'):
                    resolved_paths['.'.join(path + [k])] = v
                    resolved = resolve_ref(v)
                    res, paths = resolve_dict(resolved, path + [k])
                    d[k] = res
                    resolved_paths.update(paths)
        return d, resolved_paths
    
    for call in calls:
        res, paths = resolve_dict(call['payload'])
        call['payload'] = res
        call['__ref_paths__'] = paths
        
    return pd.DataFrame(calls)

In [5]:
# A variety of test cases, all with the the shape of:
# {'a': {'b': {'c': {'d': NUMERIC_VALUE}}}}
# but with different combinations of references

# No refs
res = insert_call({'a': {'b': {'c': {'d': 1}}}})

# Ref at A
res = insert_call({'a': insert_ref_object({'b': {'c': {'d': 2}}})})
# Ref at B
res = insert_call({'a': {'b': insert_ref_object({'c': {'d': 3}})}})
# Ref at C
res = insert_call({'a': {'b': {'c': insert_ref_object({'d': 4})}}})


# Ref at A and B
res = insert_call({'a': insert_ref_object({'b': insert_ref_object({'c': {'d': 5}})})})
# Ref at A and C
res = insert_call({'a': insert_ref_object({'b': {'c': insert_ref_object({'d': 6})}})})
# Ref at B and C
res = insert_call({'a': {'b': insert_ref_object({'c': insert_ref_object({'d': 7})})}})

# Ref at A, B and C
res = insert_call({'a': insert_ref_object({'b': insert_ref_object({'c': insert_ref_object({'d': 8})})})})

In [6]:
calls = get_calls_user_perspective()
calls

,id,payload,__ref_paths__
0,65c11804-f371-4456-8d03-2c3b8e2ccc4d,{'a': {'b': {'c': {'d': 1}}}},{}
1,68b5ff9c-f54a-4be3-87ed-2e39952107a2,{'a': {'b': {'c': {'d': 4}}}},{'a.b.c': 'ref://objects/76b739e3-08aa-4fd6-b9...
2,873cc2c3-f5be-436f-b4f7-09928b55c4fa,{'a': {'b': {'c': {'d': 6}}}},{'a': 'ref://objects/b3d2de9a-03a7-4e51-9ee9-c...
3,88ae048b-59ec-41c4-aab8-7e252d4157c2,{'a': {'b': {'c': {'d': 5}}}},{'a': 'ref://objects/91a4b25a-49fb-4ed8-84da-a...
4,cfae33db-b03f-4aba-9c4f-7907a9f259d7,{'a': {'b': {'c': {'d': 2}}}},{'a': 'ref://objects/82055cbe-7a12-447c-b8d1-c...
5,fa110a44-543d-4107-9b03-5868e7d4b846,{'a': {'b': {'c': {'d': 3}}}},{'a.b': 'ref://objects/f3b9fdbe-cea2-4e02-b64c...
6,8d288b4c-15ee-4e5e-b8a1-c335648efbf1,{'a': {'b': {'c': {'d': 8}}}},{'a': 'ref://objects/773579b1-c73d-466b-9d49-4...
7,5a9a3bd8-b138-44df-9dda-ec10e839789c,{'a': {'b': {'c': {'d': 7}}}},{'a.b': 'ref://objects/9657b55e-5e73-45f4-9882...


In [7]:
get_calls()

,id,payload_dump
0,8d288b4c-15ee-4e5e-b8a1-c335648efbf1,"{""a"": ""ref://objects/773579b1-c73d-466b-9d49-4..."
1,65c11804-f371-4456-8d03-2c3b8e2ccc4d,"{""a"": {""b"": {""c"": {""d"": 1}}}}"
2,68b5ff9c-f54a-4be3-87ed-2e39952107a2,"{""a"": {""b"": {""c"": ""ref://objects/76b739e3-08aa..."
3,873cc2c3-f5be-436f-b4f7-09928b55c4fa,"{""a"": ""ref://objects/b3d2de9a-03a7-4e51-9ee9-c..."
4,88ae048b-59ec-41c4-aab8-7e252d4157c2,"{""a"": ""ref://objects/91a4b25a-49fb-4ed8-84da-a..."
5,cfae33db-b03f-4aba-9c4f-7907a9f259d7,"{""a"": ""ref://objects/82055cbe-7a12-447c-b8d1-c..."
6,fa110a44-543d-4107-9b03-5868e7d4b846,"{""a"": {""b"": ""ref://objects/f3b9fdbe-cea2-4e02-..."
7,5a9a3bd8-b138-44df-9dda-ec10e839789c,"{""a"": {""b"": ""ref://objects/9657b55e-5e73-45f4-..."


In [8]:
get_objects()

,id,payload_dump
0,773579b1-c73d-466b-9d49-47e43eacc4f8,"{""b"": ""ref://objects/2f0364d3-eac2-4b98-96a0-a..."
1,1961e4dd-fe35-459c-9c7d-de1220fe1100,"{""d"": 7}"
2,2603cc3c-6b6a-4447-97b0-0cb13edba4d4,"{""d"": 8}"
3,2f0364d3-eac2-4b98-96a0-ad0eb8d71b86,"{""c"": ""ref://objects/2603cc3c-6b6a-4447-97b0-0..."
4,6a0a866f-97ba-49b5-89c6-f74b7ff64d5a,"{""c"": {""d"": 5}}"
5,76b739e3-08aa-4fd6-b9bf-b847cd290421,"{""d"": 4}"
6,82055cbe-7a12-447c-b8d1-cea67c3a4cb4,"{""b"": {""c"": {""d"": 2}}}"
7,91a4b25a-49fb-4ed8-84da-a451b33a784b,"{""b"": ""ref://objects/6a0a866f-97ba-49b5-89c6-f..."
8,9657b55e-5e73-45f4-9882-5e47b03742f0,"{""c"": ""ref://objects/1961e4dd-fe35-459c-9c7d-d..."
9,9936b070-21af-4cb1-8227-077519f606cf,"{""d"": 6}"


Observations:

1. The type of query we want to do requires one of two things:
    1. The query needs to support some form of recursion
    2. The data needs to be de-normalized
2. Clickhouse just added support for recursive CTEs (https://clickhouse.com/blog/clickhouse-release-24-04), but our use case is sufficiently complicated such that it seems not possible to be performant.
3. The primary drawback of de-normalization is a massive amount of data duplication.

Options:
    recursive CTE
    in-memory recursion
    de-normalization -> memory explosion
    dynamic query -

... maybe the recursion can happen at query build time since we know the depth!?!?!?!!



In [9]:
def custom_query_for_d(path):
    query_count = 0
    def build_query_for_path_part(part, subquery):
        nonlocal query_count
        query_count += 1
        subquery_name = f"subquery_{query_count}"
        return f"""
            SELECT 
                {subquery_name}.id as id,
                JSONExtractString({subquery_name}.payload_dump, '{part}') as payload_dump
            FROM
                ({subquery}) as {subquery_name}
        """
    def replace_ref_query(query):
        nonlocal query_count
        query_count += 1

        subquery_name = f"subquery_{query_count}"
        objquery_name = f"objquery_{query_count}"
        return f""" 
                SELECT 
                    {subquery_name}.id as id,
                    if({objquery_name}.payload_dump != '', {objquery_name}.payload_dump, {subquery_name}.payload_dump) as payload_dump
                FROM
                    ({query}) as {subquery_name}
                LEFT JOIN
                    objects as {objquery_name}
                ON
                    substring({subquery_name}.payload_dump, 15) = {objquery_name}.id
        """
            
            
    base_query = replace_ref_query("SELECT calls.id as id, calls.payload_dump as payload_dump FROM calls")
    for part in path:
        base_query = replace_ref_query(build_query_for_path_part(part, base_query))
    return base_query

In [10]:
client.query_df(custom_query_for_d([]))

,id,payload_dump
0,5a9a3bd8-b138-44df-9dda-ec10e839789c,"{""a"": {""b"": ""ref://objects/9657b55e-5e73-45f4-..."
1,8d288b4c-15ee-4e5e-b8a1-c335648efbf1,"{""a"": ""ref://objects/773579b1-c73d-466b-9d49-4..."
2,65c11804-f371-4456-8d03-2c3b8e2ccc4d,"{""a"": {""b"": {""c"": {""d"": 1}}}}"
3,68b5ff9c-f54a-4be3-87ed-2e39952107a2,"{""a"": {""b"": {""c"": ""ref://objects/76b739e3-08aa..."
4,873cc2c3-f5be-436f-b4f7-09928b55c4fa,"{""a"": ""ref://objects/b3d2de9a-03a7-4e51-9ee9-c..."
5,88ae048b-59ec-41c4-aab8-7e252d4157c2,"{""a"": ""ref://objects/91a4b25a-49fb-4ed8-84da-a..."
6,cfae33db-b03f-4aba-9c4f-7907a9f259d7,"{""a"": ""ref://objects/82055cbe-7a12-447c-b8d1-c..."
7,fa110a44-543d-4107-9b03-5868e7d4b846,"{""a"": {""b"": ""ref://objects/f3b9fdbe-cea2-4e02-..."


In [11]:
client.query_df(custom_query_for_d(['a']))

,id,payload_dump
0,8d288b4c-15ee-4e5e-b8a1-c335648efbf1,"{""b"": ""ref://objects/2f0364d3-eac2-4b98-96a0-a..."
1,65c11804-f371-4456-8d03-2c3b8e2ccc4d,"{""b"":{""c"":{""d"":1}}}"
2,68b5ff9c-f54a-4be3-87ed-2e39952107a2,"{""b"":{""c"":""ref://objects/76b739e3-08aa-4fd6-b9..."
3,873cc2c3-f5be-436f-b4f7-09928b55c4fa,"{""b"": {""c"": ""ref://objects/9936b070-21af-4cb1-..."
4,88ae048b-59ec-41c4-aab8-7e252d4157c2,"{""b"": ""ref://objects/6a0a866f-97ba-49b5-89c6-f..."
5,cfae33db-b03f-4aba-9c4f-7907a9f259d7,"{""b"": {""c"": {""d"": 2}}}"
6,fa110a44-543d-4107-9b03-5868e7d4b846,"{""b"":""ref://objects/f3b9fdbe-cea2-4e02-b64c-4f..."
7,5a9a3bd8-b138-44df-9dda-ec10e839789c,"{""b"":""ref://objects/9657b55e-5e73-45f4-9882-5e..."


In [12]:
client.query_df(custom_query_for_d(['a', 'b']))

,id,payload_dump
0,8d288b4c-15ee-4e5e-b8a1-c335648efbf1,"{""c"": ""ref://objects/2603cc3c-6b6a-4447-97b0-0..."
1,65c11804-f371-4456-8d03-2c3b8e2ccc4d,"{""c"":{""d"":1}}"
2,68b5ff9c-f54a-4be3-87ed-2e39952107a2,"{""c"":""ref://objects/76b739e3-08aa-4fd6-b9bf-b8..."
3,873cc2c3-f5be-436f-b4f7-09928b55c4fa,"{""c"":""ref://objects/9936b070-21af-4cb1-8227-07..."
4,88ae048b-59ec-41c4-aab8-7e252d4157c2,"{""c"": {""d"": 5}}"
5,cfae33db-b03f-4aba-9c4f-7907a9f259d7,"{""c"":{""d"":2}}"
6,fa110a44-543d-4107-9b03-5868e7d4b846,"{""c"": {""d"": 3}}"
7,5a9a3bd8-b138-44df-9dda-ec10e839789c,"{""c"": ""ref://objects/1961e4dd-fe35-459c-9c7d-d..."


In [13]:
client.query_df(custom_query_for_d(['a', 'b', 'c']))

,id,payload_dump
0,5a9a3bd8-b138-44df-9dda-ec10e839789c,"{""d"": 7}"
1,65c11804-f371-4456-8d03-2c3b8e2ccc4d,"{""d"":1}"
2,68b5ff9c-f54a-4be3-87ed-2e39952107a2,"{""d"": 4}"
3,873cc2c3-f5be-436f-b4f7-09928b55c4fa,"{""d"": 6}"
4,88ae048b-59ec-41c4-aab8-7e252d4157c2,"{""d"":5}"
5,cfae33db-b03f-4aba-9c4f-7907a9f259d7,"{""d"":2}"
6,fa110a44-543d-4107-9b03-5868e7d4b846,"{""d"":3}"
7,8d288b4c-15ee-4e5e-b8a1-c335648efbf1,"{""d"": 8}"


In [14]:
client.query_df(custom_query_for_d(['a', 'b', 'c', 'd']))

,id,payload_dump
0,8d288b4c-15ee-4e5e-b8a1-c335648efbf1,8
1,5a9a3bd8-b138-44df-9dda-ec10e839789c,7
2,65c11804-f371-4456-8d03-2c3b8e2ccc4d,1
3,68b5ff9c-f54a-4be3-87ed-2e39952107a2,4
4,873cc2c3-f5be-436f-b4f7-09928b55c4fa,6
5,88ae048b-59ec-41c4-aab8-7e252d4157c2,5
6,cfae33db-b03f-4aba-9c4f-7907a9f259d7,2
7,fa110a44-543d-4107-9b03-5868e7d4b846,3


In [15]:
client.query_df(custom_query_for_d(['a', 'b', 'c', 'd', 'e']))

,id,payload_dump
0,65c11804-f371-4456-8d03-2c3b8e2ccc4d,
1,68b5ff9c-f54a-4be3-87ed-2e39952107a2,
2,873cc2c3-f5be-436f-b4f7-09928b55c4fa,
3,88ae048b-59ec-41c4-aab8-7e252d4157c2,
4,cfae33db-b03f-4aba-9c4f-7907a9f259d7,
5,fa110a44-543d-4107-9b03-5868e7d4b846,
6,5a9a3bd8-b138-44df-9dda-ec10e839789c,
7,8d288b4c-15ee-4e5e-b8a1-c335648efbf1,


In [17]:
print(custom_query_for_d(['a', 'b', 'c', 'd']))

 
                SELECT 
                    subquery_9.id as id,
                    if(objquery_9.payload_dump != '', objquery_9.payload_dump, subquery_9.payload_dump) as payload_dump
                FROM
                    (
            SELECT 
                subquery_8.id as id,
                JSONExtractString(subquery_8.payload_dump, 'd') as payload_dump
            FROM
                ( 
                SELECT 
                    subquery_7.id as id,
                    if(objquery_7.payload_dump != '', objquery_7.payload_dump, subquery_7.payload_dump) as payload_dump
                FROM
                    (
            SELECT 
                subquery_6.id as id,
                JSONExtractString(subquery_6.payload_dump, 'c') as payload_dump
            FROM
                ( 
                SELECT 
                    subquery_5.id as id,
                    if(objquery_5.payload_dump != '', objquery_5.payload_dump, subquery_5.payload_dump) as payload_dump
           